In [3]:
import pandas as pd
import numpy as np
import requests
import json
from config import api_key
import flr_tools
import cme_tools

In [4]:
start_date = "2014-09-01"
end_date = "2018-02-04"
#query_flr_url =f"https://api.nasa.gov/DONKI/FLR?startDate={start_date}&endDate={end_date}&api_key={api_key}"

In [5]:
dont_use_nasa_time_format = False
flare_alldf = flr_tools.flare_query_all(start_date, end_date, dont_use_nasa_time_format)
flare_alldf.head()

,class_type,end_time(zulu),flare_id,linked_events,peak_time(zulu),power(w/m^2),start_time(zulu)
0,M2.5,"09/03/2014, 13:54:00",2014-09-03T13:20:00-FLR-001,[{'activityID': '2014-09-03T14:12:00-CME-001'}],"09/03/2014, 13:53:00",0.000025,"09/03/2014, 13:20:00"
1,M1.1,"09/06/2014, 17:22:00",2014-09-06T16:50:00-FLR-001,[None Found.],"09/06/2014, 17:09:00",0.000011,"09/06/2014, 16:50:00"
2,M4.6,"09/09/2014, 01:30:00",2014-09-08T23:12:00-FLR-001,[{'activityID': '2014-09-09T00:16:00-CME-001'}],"09/09/2014, 00:28:00",0.000046,"09/08/2014, 23:12:00"
3,X1.6,"09/10/2014, 18:20:00",2014-09-10T17:21:00-FLR-001,[{'activityID': '2014-09-10T18:18:00-CME-001'}...,"09/10/2014, 17:45:00",0.000160,"09/10/2014, 17:21:00"
4,M2.1,"09/11/2014, 15:31:00",2014-09-11T15:20:00-FLR-001,[None Found.],"09/11/2014, 15:25:00",0.000021,"09/11/2014, 15:20:00"


In [6]:
dont_use_nasa_time_format = False
cme_alldf = cme_tools.cme_query_all(start_date, end_date, dont_use_nasa_time_format)
cme_alldf.head()

,cme_id,speed,type,linked_events,start time
0,2014-09-01T11:24:00-CME-001,1700,O,[{'activityID': '2014-09-01T12:14:00-SEP-001'}...,"09/01/2014, 11:24:00"
1,2014-09-01T16:00:00-CME-001,900,C,[{'activityID': '2014-09-02T07:05:00-SEP-001'}...,"09/01/2014, 16:00:00"
2,2014-09-01T21:36:00-CME-001,800,C,None,"09/01/2014, 21:36:00"
3,2014-09-01T22:24:00-CME-001,1400,O,[{'activityID': '2014-09-02T07:05:00-SEP-001'}...,"09/01/2014, 22:24:00"
4,2014-09-02T02:36:00-CME-001,1140,O,[{'activityID': '2014-09-04T06:00:00-IPS-001'}],"09/02/2014, 02:36:00"


In [7]:
cme_id = []
for event in flare_alldf['linked_events'][4]:
    cme_id.append(event)

mars_search = []
[mars_search.append(flr_tools.cme_compare(id,start_date,end_date)) for id in cme_id]
mars_search     

TypeError: cme_compare() takes from 0 to 2 positional arguments but 3 were given

In [ ]:
use_nasa_time = True
flare_smalldf = flr_tools.flare_query_small(start_date, end_date, use_nasa_time)
flare_smalldf

In [ ]:
query_cme_url =f"https://kauai.ccmc.gsfc.nasa.gov/DONKI/WS/get/CME?startDate={start_date}&endDate={end_date}"
cme_request = requests.get(query_cme_url).json()

In [ ]:
cme_request

In [5]:
cme_analysis_data = flr_tools.cme_analysis_df(cme_request)
cme_analysis_data

,activity_id,speed,data_level,lat,long,type,half_angle,is_most_accurate,time21_5,note
0,2016-09-02T06:18:00-CME-001,218,0,35,-87,S,27,True,2016-09-02T19:15Z,
1,2016-09-03T12:12:00-CME-001,108,0,6,114,S,23,True,2016-09-04T18:50Z,Measurement done in SWPC cat using SOHO C2 and...
2,2016-09-04T13:25:00-CME-001,216,0,-14,26,S,32,True,2016-09-05T08:25Z,
3,2016-09-06T06:24:00-CME-001,539,0,22,160,C,10,True,2016-09-06T13:11Z,Measured using C3 and STA.
4,2016-09-06T08:54:00-CME-001,674,1,-20,120,C,31,True,2016-09-06T14:18Z,
...,...,...,...,...,...,...,...,...,...,...
318,2017-12-29T13:25:00-CME-001,303,0,19,129,S,19,True,2017-12-30T00:46Z,
319,2018-01-01T00:39:00-CME-001,279,0,21,112,S,17,True,2018-01-01T18:47Z,
320,2018-01-22T03:12:00-CME-001,560,1,-18,66,C,31,True,2018-01-22T09:28Z,The CME had an irregular form and could only b...
321,2018-01-28T18:24:00-CME-001,241,0,28,123,S,16,True,2018-01-29T09:12Z,


In [3]:
flare_cme_combo_test = flr_tools.cme_compare(start_date, end_date)

In [4]:
flare_cme_combo_test

,activity_id,speed,data_level,lat,long,type,half_angle,is_most_accurate,time21_5,note,flare_id,start_time(zulu),peak_time(zulu),end_time(zulu),class_type,power(w/m^2),linked_events
0,2014-09-03T14:12:00-CME-001,606,0,-9,-111,C,14,True,2014-09-03T20:05Z,using STEREO_cat with C3 and STA,2014-09-03T13:20:00-FLR-001,"09/03/2014, 13:20:00","09/03/2014, 13:53:00","09/03/2014, 13:54:00",M2.5,0.000025,2014-09-03T14:12:00-CME-001
1,2014-09-09T00:16:00-CME-001,780,0,26,-30,C,43,True,2014-09-09T04:16Z,Only LASCO images were available; SWPC_Cat pro...,2014-09-08T23:12:00-FLR-001,"09/08/2014, 23:12:00","09/09/2014, 00:28:00","09/09/2014, 01:30:00",M4.6,0.000046,2014-09-09T00:16:00-CME-001
2,2014-09-10T18:18:00-CME-001,1400,0,15,10,O,45,True,2014-09-10T20:00Z,Only LASCO C3 imagery available. Ambiguity in ...,2014-09-10T17:21:00-FLR-001,"09/10/2014, 17:21:00","09/10/2014, 17:45:00","09/10/2014, 18:20:00",X1.6,0.000160,2014-09-10T18:18:00-CME-001
3,2014-09-14T03:12:00-CME-001,430,0,7,32,S,45,True,2014-09-14T12:00Z,Updated analysis based on all 3 viewing angles.,2014-09-14T02:03:00-FLR-001,"09/14/2014, 02:03:00","09/14/2014, 02:16:00","09/14/2014, 02:36:00",M1.5,0.000015,2014-09-14T03:12:00-CME-001
4,2014-09-24T00:24:00-CME-001,348,0,1,-69,S,27,True,2014-09-24T11:55Z,,2014-09-23T23:02:00-FLR-001,"09/23/2014, 23:02:00","09/23/2014, 23:15:00","09/23/2014, 23:28:00",M2.3,0.000023,2014-09-24T00:24:00-CME-001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,2017-09-08T04:17:00-CME-001,442,0,-5,53,S,21,True,2017-09-08T05:29Z,,2017-09-08T03:39:00-FLR-001,"09/08/2017, 03:39:00","09/08/2017, 03:43:00","09/08/2017, 03:46:00",M1.2,0.000012,2017-09-08T04:17:00-CME-001
97,2017-09-08T07:24:00-CME-001,450,0,-3,54,S,20,True,2017-09-08T14:03Z,,2017-09-08T07:40:00-FLR-001,"09/08/2017, 07:40:00","09/08/2017, 07:49:00","09/08/2017, 07:58:00",M8.1,0.000081,2017-09-08T07:24:00-CME-001
98,2017-09-10T16:09:00-CME-001,2650,1,-12,85,R,54,True,2017-09-10T17:18Z,,2017-09-10T15:35:00-FLR-001,"09/10/2017, 15:35:00","09/10/2017, 16:06:00","09/10/2017, 16:31:00",X8.2,0.000820,2017-09-10T16:09:00-CME-001
99,2017-09-12T08:09:00-CME-001,450,0,-9,-36,S,22,True,2017-09-12T16:30Z,Measure STEREO_CAT to velocities <400 km/s.,2017-09-12T07:23:00-FLR-001,"09/12/2017, 07:23:00","09/12/2017, 07:29:00","09/12/2017, 07:48:00",C3.0,0.000003,2017-09-12T08:09:00-CME-001


In [6]:
len(flare_cme_combo_test)

305545

In [6]:
cme_ids = cme_tools.linked_events(flare_alldf.rename(columns={'flare_id':'id'}),'CME')
flr_ids = cme_tools.linked_events(cme_alldf.rename(columns={'cme_id':'id'}),'FLR')
both = pd.DataFrame(cme_ids, columns=['flare_id','cme_id'])
new_df = both.merge(cme_alldf[["cme_id","speed","type","start time"]], on="cme_id", how='left')
new_df = new_df.rename(columns={"speed":"cme speed","type": "cme type", "start time": "cme start time"})
full_df = new_df.merge(flare_alldf[["flare_id","start_time(zulu)","peak_time(zulu)","end_time(zulu)","class_type","power(w/m^2)"]],
                      on="flare_id", how="left")
full_df = full_df.rename(columns={"start_time(zulu)":"flare start time","peak_time(zulu)": "flare peak time",
                                  "end_time(zulu)":"flare end time","class_type":"flare type","power(w/m^2)":"flare power(w/m^)"})
full_df.head()

,flare_id,cme_id,cme speed,cme type,cme start time,flare start time,flare peak time,flare end time,flare type,flare power(w/m^)
0,2014-09-03T13:20:00-FLR-001,2014-09-03T14:12:00-CME-001,606,C,"09/03/2014, 14:12:00","09/03/2014, 13:20:00","09/03/2014, 13:53:00","09/03/2014, 13:54:00",M2.5,0.000025
1,2014-09-08T23:12:00-FLR-001,2014-09-09T00:16:00-CME-001,780,C,"09/09/2014, 00:16:00","09/08/2014, 23:12:00","09/09/2014, 00:28:00","09/09/2014, 01:30:00",M4.6,0.000046
2,2014-09-10T17:21:00-FLR-001,2014-09-10T18:18:00-CME-001,1400,O,"09/10/2014, 18:18:00","09/10/2014, 17:21:00","09/10/2014, 17:45:00","09/10/2014, 18:20:00",X1.6,0.000160
3,2014-09-14T02:03:00-FLR-001,2014-09-14T03:12:00-CME-001,430,S,"09/14/2014, 03:12:00","09/14/2014, 02:03:00","09/14/2014, 02:16:00","09/14/2014, 02:36:00",M1.5,0.000015
4,2014-09-23T23:02:00-FLR-001,2014-09-24T00:24:00-CME-001,348,S,"09/24/2014, 00:24:00","09/23/2014, 23:02:00","09/23/2014, 23:15:00","09/23/2014, 23:28:00",M2.3,0.000023
